In [ ]:
pip install SoundFile


**Importing Necessary Libraries and Declaring Global Variables Cell:**

In [ ]:
from tkinter import filedialog
import soundfile as sf
import numpy as np
from scipy.fft import fft, fftfreq
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from scipy import signal
from tkinter import messagebox
import cmath



global win_options
global whole_layout
global win_len
global chosen_len
global compare_mode_choice
global calc_mode_choice
global seg_no
global N_dft_entry
global audio_len
global same_win_md_flg
global win_compare_len
global win_compare_winds
global sampled_audio_abs_fft
global freq_axs
global chosen_format
global whc_to_plt_flg
global applied_win

same_win_md_flg = 0
diff_win_md_flg = 0
whc_to_plt_flg = 0

****Functions' Definition Cell:****

In [ ]:
#This function is set-up to upload and read the chosen audio file, whenever the user clicks on the upload audio button.
def UploadAudio():
    global sampled_audio, sampling_rate, audio_len,  sampled_audio_abs_fft
    filename = filedialog.askopenfilename() #prompt a window to select your audio file
    sampled_audio, sampling_rate = sf.read(filename) #reading your audio file returns the audio file sampled along with the sampling rate 
    audio_len = len(sampled_audio)
    sampled_audio_abs_fft = np.abs(np.fft.fft(sampled_audio))
#================================================================================================================= 
    
#This function takes in a window function and audio signal amplitudes and returns the windowed audio signal for more spectrul clarity.    
def applyWindow(user_win, audio_sig):
      N = len(user_win)
      Windowed_time = np.zeros(N, dtype=float)
  
      for k in range(chosen_len):
          Windowed_time[k] = Windowed_time[k] + (user_win[k]*audio_sig[k])

      return Windowed_time

#================================================================================================================= 

#It compares the length of the window function to the the length of the audio signal and returns a zero-padded signal in case the 
#length of the window is greater than the audio signal, ohterwise, it returns the original audio signal.
def compare_win_audio_len(aud_len, wind_len):
    if aud_len < wind_len:
        samp_aud = np.pad(sampled_audio, (0, wind_len - aud_len), 'constant', constant_values =0)
        return samp_aud
    else:
        return sampled_audio

#================================================================================================================= 

#This function gets the the windowed signals plotted both in the time- and frequency-domain.
def plotWindowed(t,sam):
    plt_time_freq = tk.Tk()
    freq_axs1 = fftfreq(len(sam), 1 / sampling_rate)
    plt_time_freq.title('Time- vs Frequency-domain representation of the Windowed Signal')
    figure = plt.Figure(figsize=(6,5), dpi=100)
    ax = figure.add_subplot(111)
    ax.plot(t,sam)
    ax.set_xlabel('Time')
    ax.set_ylabel('Signal Amplitude')
    canvas = FigureCanvasTkAgg(figure, master = plt_time_freq)   
    canvas.draw() 
    canvas.get_tk_widget().pack() 
    
    figure2 = plt.Figure(figsize=(6,5), dpi=100)
    ax2 = figure2.add_subplot(111)
    ax2.plot(freq_axs1, np.abs(np.fft.fft(sam)))
    ax2.set_xlabel('Frequency')
    ax2.set_ylabel('Amplitude')    
    canvas2 = FigureCanvasTkAgg(figure2, master = plt_time_freq)   
    canvas2.draw() 
    canvas2.get_tk_widget().pack()

#================================================================================================================= 
   
#Here, this modulde apply the window function chosen by the user to the audio signal and generate time- and frequency-domain of the
#resultant windowed signal.
def applyChosenWindow(event):
    global applied_win 
    global chosen_len
    #chosen_len = int(win_len.get())
    if same_win_md_flg == 0:
        chosen_len = int(win_len.get())
        whc_to_plt_flg = 1
    elif same_win_md_flg == 1:    
        chosen_len = int(win_compare_len.get())
        
    if win_options.get() == 'Triangular Window':
        chosen_window_func = signal.triang(int(chosen_len))
        nw_audio = compare_win_audio_len(audio_len, chosen_len)
        applied_win = applyWindow(chosen_window_func, nw_audio)
        time = np.linspace(0,7,len(applied_win))
        plotWindowed(time,applied_win)
        
        
    elif win_options.get() == 'Hann Window':
        chosen_window_func = signal.hann(int(chosen_len))
        nw_audio = compare_win_audio_len(audio_len, chosen_len)
        applied_win = applyWindow(chosen_window_func, nw_audio)
        time = np.linspace(0,20,len(applied_win))
        plotWindowed(time,applied_win)

    elif win_options.get() == 'Hamming Window':
        chosen_window_func = signal.hamming(int(chosen_len))
        nw_audio = compare_win_audio_len(audio_len, chosen_len)
        applied_win = applyWindow(chosen_window_func, nw_audio)
        time = np.linspace(0,20,len(applied_win))
        plotWindowed(time,applied_win)

    elif win_options.get() == 'Rectangular Window':
        chosen_window_func = signal.boxcar(int(chosen_len))
        nw_audio = compare_win_audio_len(audio_len, chosen_len)
        applied_win = applyWindow(chosen_window_func, nw_audio)
        time = np.linspace(0,20,len(applied_win))
        plotWindowed(time,applied_win)
    
#================================================================================================================= 

#Once the 'Get Time- and Frequency-domain Plots of the Signal' button is clicked, this function generates time- 
#and frequency-domain plots of the signal  
def PltTFD_Sig():    
    global freq_axs
    plt_time_freq = tk.Tk()
    duration = len(sampled_audio)/sampling_rate
    time_axs = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
    freq_axs = fftfreq(len(sampled_audio), 1 / sampling_rate)
    dft_audio = np.abs(fft(sampled_audio, n =len(sampled_audio)))
    
    plt_time_freq.title('Time- vs Frequency-domain representation of the Signal')
    figure = plt.Figure(figsize=(6,5), dpi=100)
    ax = figure.add_subplot(111)
    ax.plot(time_axs,sampled_audio)
    ax.set_xlabel('Time')
    ax.set_ylabel('Signal Amplitude')
    canvas = FigureCanvasTkAgg(figure, master = plt_time_freq)   
    canvas.draw() 
    canvas.get_tk_widget().pack() 
    
    figure2 = plt.Figure(figsize=(6,5), dpi=100)
    ax2 = figure2.add_subplot(111)
    ax2.plot(freq_axs,dft_audio)
    ax2.set_xlabel('Frequency')
    ax2.set_ylabel('Amplitude')    
    canvas2 = FigureCanvasTkAgg(figure2, master = plt_time_freq)   
    canvas2.draw() 
    canvas2.get_tk_widget().pack()

#================================================================================================================= 

#this function prompts both a label and combobox to enable the user to choose the window length,
#whenever the the window function is chosen.  
def GetWinLen(event):
    global win_len

    ttk.Label(whole_layout, text = "Choose the window length:       ",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 17, padx = 10, pady = 25)
    n = tk.StringVar() 
    win_len = ttk.Combobox(whole_layout, width = 27, textvariable = n) 
    win_len['values'] = ('','441000','148000', '304999', '1110000') 
    win_len.grid(column = 1, row = 17) 
    win_len.current(0)
    win_len.bind("<<ComboboxSelected>>", applyChosenWindow)

#================================================================================================================= 

#If the user wishes to compare the intially-chosen window to a different window of his/her choice, this function apply the window 
#function to the signal and plot it.
def applyChosenWindowCompare(event = None):
        
        if win_compare_winds.get() == 'Triangular Window':
            chosen_window_func = signal.triang(int(chosen_len))
            nw_audio = compare_win_audio_len(audio_len, chosen_len)
            applied_win2 = applyWindow(chosen_window_func, nw_audio)
            duration = len(applied_win2)/sampling_rate
            time_axs = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
            plotWindowed(time_axs,applied_win2)
        
        
        elif win_compare_winds.get() == 'Hann Window':
            chosen_window_func = signal.hann(int(chosen_len))
            nw_audio = compare_win_audio_len(audio_len, chosen_len)
            applied_win2 = applyWindow(chosen_window_func, nw_audio)
            duration = len(applied_win2)/sampling_rate
            time_axs = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
            plotWindowed(time_axs,applied_win2)
            
        elif win_compare_winds.get() == 'Hamming Window':
            chosen_window_func = signal.hamming(int(chosen_len))
            nw_audio = compare_win_audio_len(audio_len, chosen_len)
            applied_win2 = applyWindow(chosen_window_func, nw_audio)
            duration = len(applied_win2)/sampling_rate
            time_axs = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
            plotWindowed(time_axs,applied_win2)
            
        elif win_compare_winds.get() == 'Rectangular Window':
            chosen_window_func = signal.boxcar(int(chosen_len))
            nw_audio = compare_win_audio_len(audio_len, chosen_len)
            applied_win2 = applyWindow(chosen_window_func, nw_audio)
            duration = len(applied_win2)/sampling_rate
            time_axs = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
            plotWindowed(time_axs,applied_win2)

#================================================================================================================= 
    
#This function enables the user to choose a comparison mode by displaying a combobox to choose the desired mode.    
def compareMode(event):
    global same_win_md_flg, win_compare_len, win_compare_winds
    
    if compare_mode_choice.get() == 'Different Window':
        n = tk.StringVar() 
        ttk.Label(whole_layout, text = "Choose another window for comparison:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 19, padx = 10, pady = 25)
        win_compare_winds = ttk.Combobox(whole_layout, width = 27, textvariable = n) 
        win_compare_winds['values'] = ('','Triangular Window','Hamming Window', 'Hann Window', 'Rectangular Window') 
        win_compare_winds.grid(column = 1, row = 19) 
        win_compare_winds.current(0)
        win_compare_winds.bind("<<ComboboxSelected>>", applyChosenWindowCompare)

    elif compare_mode_choice.get() == 'Same Window, different length':
        ttk.Label(whole_layout, text = "Choose window length for comparison:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 19, padx = 10, pady = 25) 
        n = tk.StringVar() 
        win_compare_len = ttk.Combobox(whole_layout, width = 27, textvariable = n) 
        win_compare_len['values'] = ('','441000','148000', '304999', '1110000') 
        win_compare_len.grid(column = 1, row = 19) 
        win_compare_len.current(0)
        same_win_md_flg = 1
        win_compare_len.bind("<<ComboboxSelected>>", applyChosenWindow)


#================================================================================================================= 

#This function is called whenever the user chooses the number of segments to calculate the RMS averging.   
def RMSaverage(event): 
        segments_ls = list()
        seg_num = int(seg_no.get())
        samples_no_in_seg = int(np.floor(len(sampled_audio_abs_fft)/seg_num))

        indx = 0
        for i in range(seg_num):
            segm = sampled_audio_abs_fft[indx:(((i+1)*samples_no_in_seg))]
            segments_ls.append(segm)
            indx = indx + samples_no_in_seg
        #segments_ls.append(sampled_audio[indx:indx+len(sampled_audio)])
        fft_avg_ls = list()
        for j in range(samples_no_in_seg):
            aver = 0
            for i in range(seg_num):
                aver = np.abs(segments_ls[i][j]) + aver

            rms_avg = aver/seg_num
            fft_avg_ls.append(rms_avg)
        freq_bins = list(range(len(fft_avg_ls)))

        plt_time_freq = tk.Tk()
        plt_time_freq.title("RMS Averaging Calculation Mode")
        figure2 = plt.Figure(figsize=(6,5), dpi=100)
        ax2 = figure2.add_subplot(111)
        ax2.plot(freq_bins, fft_avg_ls)
        ax2.set_xlabel('Frequency Bins')
        ax2.set_ylabel('Amplitude')    
        canvas2 = FigureCanvasTkAgg(figure2, master = plt_time_freq)   
        canvas2.draw() 
        canvas2.get_tk_widget().pack()

#================================================================================================================= 

#This function is called whenever the user clicks on the 'GET POWER' button to calculate the power within a frequency span
#specified by the user.
def avgPowerOverSpan(event = None):
        sum_square = 0
        fft_ls = list(freq_axs)
        span1_input = fft_ls.index(float(span1.get())) #Get the index of the given freq
        span2_input = fft_ls.index(float(span2.get()))
        
        sampled_audio_ovr_span = sampled_audio_abs_fft[span1_input: span2_input]
        
        for i in range(len(sampled_audio_ovr_span)):
            sum_square = sum_square + np.power(sampled_audio_ovr_span[i],2)
        
        avg_power = sum_square/len(sampled_audio_ovr_span) 
        messagebox.showinfo("The Average Power of the Audio Signal", f"The Average Power of the Audio Signal is {avg_power}")

#================================================================================================================= 

#This function displays two frequecy entries and a button, in the case of the user's choice is to get the power over the span specified by the 
#entered frequencies. Otherwise, it displays a combobox for the user to choose the number of segments for the RMS calculations.
def calcMode(event):
     global seg_no, span1, span2
    
     if calc_mode_choice.get() =='Power over certain frequency span':
        ttk.Label(whole_layout, text = "Enter the limits of the frequecny span:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 21, padx = 10, pady = 25) 
        
        span1 = tk.Entry(whole_layout)
        span1.grid(row=21, column=1)
        
        span2 = tk.Entry(whole_layout)
        span2.grid(row=21, column=2)
        
        button5 = tk.Button(whole_layout, text = 'GET POWER', command = avgPowerOverSpan)
        button5.grid(row = 21, column = 3)
        
     elif calc_mode_choice.get() == 'RMS averaging':
        ttk.Label(whole_layout, text = "Choose the number of segments:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 21, padx = 10, pady = 25) 
        n3 = tk.StringVar() 
        seg_no = ttk.Combobox(whole_layout, width = 27, textvariable = n3) 
        seg_no['values'] = ('', '64', '128', '512') 
        seg_no.grid(column = 1, row = 21) 
        seg_no.current(0)
        seg_no.bind("<<ComboboxSelected>>", RMSaverage)

#================================================================================================================= 
        
#This function get the frequencies within which the user wishes to examine the spectrum. And, plot the spectrum within those frequencies.        
def pltFreqSpan(event = None):
        fft_ls = list(freq_axs)
        span1_input = fft_ls.index(float(bin1.get())) #Get the index of the given freq
        span2_input = fft_ls.index(float(bin2.get()))
       
        plt_time_freq = tk.Tk()
        figure2 = plt.Figure(figsize=(6,5), dpi=100)
        ax2 = figure2.add_subplot(111)
        ax2.plot(freq_axs[span1_input:span2_input],sampled_audio_abs_fft[span1_input: span2_input])
        ax2.set_xlabel('Frequency')
        ax2.set_ylabel('Amplitude')    
        canvas2 = FigureCanvasTkAgg(figure2, master = plt_time_freq)   
        canvas2.draw() 
        canvas2.get_tk_widget().pack()
#===================================================================================================================
#After choosing the window function and its length, the function is called to view the frequency-domain of the windowed signal 
#in different formats.
def obtainFormat(event = None):
       if chosen_format.get() == 'Linear and logarithmic':
            freq_axs1 = fftfreq(len(applied_win), 1 / sampling_rate)
            
            plt_time_freq = tk.Tk()
            figure2 = plt.Figure(figsize=(6,5), dpi=100)
            ax2 = figure2.add_subplot(111)
            ax2.set_title("Logarithmic-scale Plot")
            ax2.set_yscale("log")
            ax2.plot(freq_axs1, np.abs(np.fft.fft(applied_win)))
            ax2.set_xlabel('Frequency')
            ax2.set_ylabel('Amplitude')    
            canvas2 = FigureCanvasTkAgg(figure2, master = plt_time_freq)   
            canvas2.draw() 
            canvas2.get_tk_widget().pack()
            
            figure3 = plt.Figure(figsize=(6,5), dpi=100)
            ax3 = figure3.add_subplot(111)
            ax3.set_title("Linear-scale Plot")
            ax3.plot(freq_axs1, np.abs(np.fft.fft(applied_win)))
            ax3.set_xlabel('Frequency')
            ax3.set_ylabel('Amplitude')    
            canvas3 = FigureCanvasTkAgg(figure3, master = plt_time_freq)   
            canvas3.draw() 
            canvas3.get_tk_widget().pack()
            
       elif chosen_format.get() == 'Magnitude and phase':
            magintude = np.abs(np.fft.fft(applied_win))
            fft_only = np.fft.fft(applied_win)
            freq_axs1 = fftfreq(len(applied_win), 1 / sampling_rate)
            phase_list = list()
            
            for i in range(len(fft_only)):
                l = cmath.phase(fft_only[i])
                phase_list.append(l)
                
            plt_time_freq = tk.Tk()
            figure2 = plt.Figure(figsize=(6,5), dpi=100)
            ax2 = figure2.add_subplot(111)
            ax2.set_title("Magnitude Plot")
            ax2.plot(freq_axs1, magintude)
            ax2.set_xlabel('Frequency')
            ax2.set_ylabel('Amplitude')    
            canvas2 = FigureCanvasTkAgg(figure2, master = plt_time_freq)   
            canvas2.draw() 
            canvas2.get_tk_widget().pack()
                       
            figure3 = plt.Figure(figsize=(6,5), dpi=100)
            ax3 = figure3.add_subplot(111)
            ax3.set_title("Phase Plot")
            ax3.plot(freq_axs1, phase_list)
            ax3.set_xlabel('Frequency')
            ax3.set_ylabel('Amplitude')    
            canvas3 = FigureCanvasTkAgg(figure3, master = plt_time_freq)   
            canvas3.draw() 
            canvas3.get_tk_widget().pack()
       elif chosen_format.get() ==  'Real and imaginary':
            fft_only = np.fft.fft(applied_win)
            freq_axs1 = fftfreq(len(applied_win), 1 / sampling_rate)
            
            real = fft_only.real
            imaginary = fft_only.imag
            
            plt_time_freq = tk.Tk()
            figure2 = plt.Figure(figsize=(6,5), dpi=100)
            ax2 = figure2.add_subplot(111)
            ax2.set_title("Real Part Plot")
            ax2.plot(freq_axs1, real)
            ax2.set_xlabel('Frequency')
            ax2.set_ylabel('Amplitude')    
            canvas2 = FigureCanvasTkAgg(figure2, master = plt_time_freq)   
            canvas2.draw() 
            canvas2.get_tk_widget().pack()
                       
            figure3 = plt.Figure(figsize=(6,5), dpi=100)
            ax3 = figure3.add_subplot(111)
            ax3.set_title("Imaginary Part Plot")
            ax3.plot(freq_axs1, imaginary)
            ax3.set_xlabel('Frequency')
            ax3.set_ylabel('Amplitude')    
            canvas3 = FigureCanvasTkAgg(figure3, master = plt_time_freq)   
            canvas3.draw() 
            canvas3.get_tk_widget().pack()
#====================================================================================================================
#After the user enters the N for DFT application, this function is called to get a portion of the signal with length N to apply 
#the DFT  on, and plot the frequency-domain of that portion to examine the resolution.
def NDFT(event = None):
            N = int(N_dft_entry.get())
            signal_pt = sampled_audio[:N] 
            fft_sig = np.abs(np.fft.fft(signal_pt))
            freq_axs3 =  fftfreq(len(fft_sig), 1 / sampling_rate) 

            plt_time_freq = tk.Tk()
            figure2 = plt.Figure(figsize=(6,5), dpi=100)
            ax2 = figure2.add_subplot(111)
            ax2.set_title("N-DFT Plot")
            ax2.plot(freq_axs3, fft_sig)
            ax2.set_xlabel('Frequency')
            ax2.set_ylabel('Amplitude')    
            canvas2 = FigureCanvasTkAgg(figure2, master = plt_time_freq)   
            canvas2.draw() 
            canvas2.get_tk_widget().pack()      

**The Program Layout Cell:**

In [ ]:
#This cell presents the whole layout of the program

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

whole_layout = tk.Tk()
whole_layout.geometry('800x800')
whole_layout.title('Spectrum Analayzer')

#A guiding label for the user to upload the audio file, a 'UPLOAD .WAV' button when clicked, the user is enabled to select the audio file
# and another 'Get Time- and Frequency- domain Plots of the Signal' button to get the plots of the original signal in both the time- and
#the frequency- domain
ttk.Label(whole_layout, text = "Upload your .WAV audio file:       ",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 15, padx = 10, pady = 25) 

button = tk.Button(whole_layout, text = 'UPLOAD .WAV', command = UploadAudio)
button.grid(row = 15, column = 1) 

button1 = tk.Button(whole_layout, text = 'Get Time- and Frequency-\ndomain Plots of the Signal', command = PltTFD_Sig)
button1.grid(row = 15, column = 2) 

#==============================================================================================================

#A guiding label for the user to select a window function, a combobox to display the window options and another combobox to 
#allow the user to choose a format for plotting the windowed signal.
ttk.Label(whole_layout, text = "Choose a window function and plotting format:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 16, padx = 10, pady = 25) 
n = tk.StringVar() 
win_options = ttk.Combobox(whole_layout, width = 27, textvariable = n) 
win_options['values'] = ('','Triangular Window','Hamming Window', 'Hann Window', 'Rectangular Window') 
win_options.grid(column = 1, row = 16) 
win_options.current(0)
win_options.bind("<<ComboboxSelected>>", GetWinLen)

n1 = tk.StringVar() 
chosen_format = ttk.Combobox(whole_layout, width = 27, textvariable = n1) 
chosen_format['values'] = ('', 'Linear and logarithmic', 'Magnitude and phase','Real and imaginary') 
chosen_format.grid(column = 3, row = 16) 
chosen_format.current(0)
chosen_format.bind("<<ComboboxSelected>>", obtainFormat)
#==============================================================================================================`

#A guiding label for the user to select a mode for comparison, either with a different window or with the same window selected 
#before of different length. A combobox to display options for the user.
ttk.Label(whole_layout, text = "Choose mode for comparison:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 18, padx = 10, pady = 25) 
n2 = tk.StringVar() 
compare_mode_choice = ttk.Combobox(whole_layout, width = 27, textvariable = n2) 
compare_mode_choice['values'] = ('','Different Window','Same Window, different length') 
compare_mode_choice.grid(column = 1, row = 18) 
compare_mode_choice.current(0)
compare_mode_choice.bind("<<ComboboxSelected>>", compareMode)

#==============================================================================================================
#A guiding label for the user to select a mode for calculation, either calculate the average power of a potion of the signal or the
#RMS averaging of the signal. A combobox to display options for the user.
ttk.Label(whole_layout, text = "Choose mode for calculations:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 20, padx = 10, pady = 25) 
n3 = tk.StringVar() 
calc_mode_choice = ttk.Combobox(whole_layout, width = 27, textvariable = n3) 
calc_mode_choice['values'] = ('', 'Power over certain frequency span', 'RMS averaging') 
calc_mode_choice.grid(column = 1, row = 20) 
calc_mode_choice.current(0)
calc_mode_choice.bind("<<ComboboxSelected>>", calcMode)


#==============================================================================================================

#A guiding label for the user to enter the length of the potion of the audio signal he/she wishes to calculate the DFT for, and 
#a button to be clicked when the user enters N.
ttk.Label(whole_layout, text = "Enter N for N-DFT (make sure it's a power of 2):",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 22, padx = 10, pady = 25) 
N_dft_entry = tk.Entry(whole_layout)
N_dft_entry.grid(row=22, column=1)

button2 = tk.Button(whole_layout, text = 'GET DFT', command = NDFT)
button2.grid(row = 22, column = 2) 

#==============================================================================================================
#A guiding label for the user to enter the limits of the frequency span the user wishes to view the frequency-domain within. And, a 
#button to be clicked when the user enters the limits of the frequency span.
ttk.Label(whole_layout, text = "View signal in frequency span within:",font = ("Times New Roman", 10)).grid(column = 0,  
        row = 23, padx = 10, pady = 25)
bin1 = tk.Entry(whole_layout)
bin1.grid(row=23, column=1)

bin2 = tk.Entry(whole_layout)
bin2.grid(row=23, column=2)

button3 = tk.Button(whole_layout, text = 'GET FREQ SPAN GRAPH', command = pltFreqSpan)
button3.grid(row = 23, column = 3)
whole_layout.mainloop()

**For testing purposes, these two lists below contains frequencies you must use if you want to view the frequency-domain signal with certain frequency span, or  for average power calculations.**

In [ ]:
#freuencies for "listen-to-the-ancient-egyptions-tv" audio file.
ls2=list(freq_axs)
ls2

[0.0,
 0.03772137733322436,
 0.07544275466644872,
 0.11316413199967307,
 0.15088550933289743,
 0.1886068866661218,
 0.22632826399934614,
 0.26404964133257053,
 0.30177101866579487,
 0.3394923959990192,
 0.3772137733322436,
 0.41493515066546793,
 0.45265652799869227,
 0.49037790533191666,
 0.5280992826651411,
 0.5658206599983654,
 0.6035420373315897,
 0.6412634146648141,
 0.6789847919980384,
 0.7167061693312629,
 0.7544275466644872,
 0.7921489239977115,
 0.8298703013309359,
 0.8675916786641602,
 0.9053130559973845,
 0.943034433330609,
 0.9807558106638333,
 1.0184771879970578,
 1.0561985653302821,
 1.0939199426635065,
 1.1316413199967308,
 1.1693626973299551,
 1.2070840746631795,
 1.2448054519964038,
 1.2825268293296281,
 1.3202482066628525,
 1.3579695839960768,
 1.3956909613293011,
 1.4334123386625257,
 1.47113371599575,
 1.5088550933289744,
 1.5465764706621987,
 1.584297847995423,
 1.6220192253286474,
 1.6597406026618717,
 1.697461979995096,
 1.7351833573283204,
 1.7729047346615447,
 1

In [ ]:
#frquencies for "C1000-17" audio file.
ls1 = list(freq_axs)
ls1

[0.0,
 0.03772137733322436,
 0.07544275466644872,
 0.11316413199967307,
 0.15088550933289743,
 0.1886068866661218,
 0.22632826399934614,
 0.26404964133257053,
 0.30177101866579487,
 0.3394923959990192,
 0.3772137733322436,
 0.41493515066546793,
 0.45265652799869227,
 0.49037790533191666,
 0.5280992826651411,
 0.5658206599983654,
 0.6035420373315897,
 0.6412634146648141,
 0.6789847919980384,
 0.7167061693312629,
 0.7544275466644872,
 0.7921489239977115,
 0.8298703013309359,
 0.8675916786641602,
 0.9053130559973845,
 0.943034433330609,
 0.9807558106638333,
 1.0184771879970578,
 1.0561985653302821,
 1.0939199426635065,
 1.1316413199967308,
 1.1693626973299551,
 1.2070840746631795,
 1.2448054519964038,
 1.2825268293296281,
 1.3202482066628525,
 1.3579695839960768,
 1.3956909613293011,
 1.4334123386625257,
 1.47113371599575,
 1.5088550933289744,
 1.5465764706621987,
 1.584297847995423,
 1.6220192253286474,
 1.6597406026618717,
 1.697461979995096,
 1.7351833573283204,
 1.7729047346615447,
 1